# Pre-flight check in Python

The following steps demonstrate how to create a first Spark app, written in Python

## Step 1: Create a collection of integers in the range of 1 .. 10000

In [2]:
data = xrange(1, 10001)

That creates a collection in Python -- no Spark yet...

## Step 2: Use that collection to create a base RDD

In [3]:
distData = sc.parallelize(data)

**RDD** is the basic data abstraction in Spark. It stands for **Resilient Distributed Dataset**. Think of it as a collection that's been distributed across the cluster. We can apply different functions to those RDDs which is where the notion of closures and functional programming comes in.

We are using a built-in handle called `sc` which stands for **SparkContext**. Typically when you write a standalone Spark app, the first thing it will do is create a SparkContext. That keeps track of dependency injection. It's got configuration, it knows what cluster resources you are using and all the API hangs off of SparkContext.

Spark Streaming and SparkSQL will subclass the SparkContext and have their own richer set of Context.

`sc.parallelize()` takes the Python collection and converts it to a RDD. `distData` essentially is just a *pointer* to the RDD. And it begins to build out what we call a **lineage** or a **operator** graph. It builds a graph of what needs to be computed. As we apply more and more functions, we extend out this graph. When we finally get to a point of needing a piece of data then that graph will be executed out on the cluster.

This is a process called **lazy evaluation**. We don't execute the commands right away rather we defer that execution so that we can look at the larger scope of what needs to be done. That allows for more optimization, fault tolerance, optimization etc. 

## Step 3: Apply functions to the RDD to define a workflow

Namely a `filter()` tranformation to keep the values less than 10, then a `collect()` action to collect the results.

In [4]:
distData.filter(lambda x: x < 10).collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9]

`collect()` is what we call an **action**. Whenever an action is called, that's when the graph gets evaluated on the cluster. 

Those three elements will show up again in just about every Spark app:

* create a *base RDD*
* apply [transformations](https://spark.apache.org/docs/latest/programming-guide.html#transformations)
* apply [actions](https://spark.apache.org/docs/latest/programming-guide.html#actions)

`filter()` here is a **closure**. Anything that is defined inside the closure is what gets put into the graph. 

The operator graph is what is called a **DAG** or Directed Acyclic Graph. The idea is that you start from some sources of data and apply some operations like joins, filters, selects until you get to the results you want. The graph is essentially a *logical layer representation* of a query. When the system goes to optimize this it has to plan and come up with a *physical representation* of the query.

In Spark, the graph gets sent out to every node. So every node will get a copy of the graph and it will execute it on it's own data partition. This allows for fault tolerance since if a node fails, we can simply transfer it's partition to some other node and that node can then execute it's copy of the graph on the partition. With this abstraction, we can achieve parallelism and fault tolerance and also be able to do a lot more optimization.

In Spark, it's more a matter of moving the code to the data than the other way around. In reality the nodes in the cluster we are talking about have a lot of resources in terms of disks, SSDs, memory, multicore etc. The key for Spark is leveraging these resources effectively.

In Operating Systems, there is a notion of **working set theory** being able to get just the right amount of memory being used. There is a similar notion here. So as we are evaluating the graph, there are ways to take and persist the intermediate calculations. Take the data you are using multiple times and cache it in memory so that we can access it at memory speeds as compared to getting it from disk speeds.

When you are doing some kind of join or aggregation, there is going to be a need to Shuffle. That's where the data is being redistributed across the cluster. That's one thing we want to limit or make as efficient as possible. So two critical questions when tuning a Spark app is *How to shuffle?* and *How to serialize?*